# codecentric.AI Bootcamp - Gradient Boosting & XGBoost

Hallo und herzlich Willkommen zurück beim **codecentric.AI Bootcamp**.

Heute geht es wieder um die Grundlagen des maschinellen Lernens, nämlich um **Gradient Boosting** und **XGBoost**. Dieses Notebook enthält Beispiele und Übungsaufgaben.

Eine theoretische Einführung in Gradient Boosting gibt es in diesem [YouTube video](https://youtu.be/xXZeVKP74ao).

In [1]:
# lade Video
from IPython.display import IFrame    
IFrame('https://www.youtube.com/embed/xXZeVKP74ao', width=850, height=650)

Zu diesem Notebook gibt es ebenfalls ein [Video](https://youtu.be/4oLsev95Lh4), in dem ich euch durch dieses Beispiel durchführe.

In [ ]:
# lade Video
from IPython.display import IFrame    
IFrame('https://www.youtube.com/embed/4oLsev95Lh4', width=850, height=650)

## Bibliotheken

Zunächst laden wir die grundlegenden Pakete, die wir für die Vorbereitung der Daten benötigten. Dazu gehören

- **numpy**: NumPy ist das wichtigste Paket für maschinelles Lernen in Python, denn es bietet die nötigen Funktionen für die Arbeit mit Matrizen und n-dimensionalen Arrays, linearer Algebra, und mehr.
- **pandas**: pandas erleichtert das Arbeiten mit Daten in Python.
- **matplotlib**: Zum Erstellen von Graphiken und Abbildungen aus unseren Daten nutzen wir matplotlib. Den zusätzlichen Befehl `matplotlib inline` geben wir in unserem Juypter Notebook mit, damit wir die generierten Plots unterhalb des Code-Chunks sehen können.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Daten einlesen und vorbereiten

Genau wie [Random Forest](https://www.youtube.com/embed/ieF_QjVUNEQ), ist Gradient Boosting einer von vielen **Machine Learning** Algorithmen des **überwachten Lernens**, im Englischen "supervised learning" genannt.

Darum nutzen wir hier denselben Datensatz, wie in dem [Random Forest](https://www.youtube.com/embed/ieF_QjVUNEQ) Kapitel.

Bei überwachtem Lernen nutzen wir sogenannte gelabelte **Trainingsdaten**.
Das bedeutet, dass wir für jeden unseren Datenpunkte ein bekanntes Ergebnis als
Zielgröße haben. Die Maschine lernt nun also dieses bekannte Ergebnis möglichst gut mit
den vorhandenen Daten abzubilden - sie lernt quasi die optimale mathematische 
Repräsentation der Daten um mit möglichst hoher Genauigkeit auf die Zielgröße zu kommen.
Die gelernte mathematische Repräsentation kann dann auf neue - ungelabelte - Daten
angewandt werden und so für Vorhersagen genutzt werden.

Das Scikit-learn Paket beinhaltet eine Reihe von **Datensätzen**, die wir direkt einladen und nutzen können. Eine Übersicht über die enthaltenen Datensätze ist hier zu finden: http://scikit-learn.org/stable/datasets/index.html

Hier wollen wir einen Datensatz über **Weinqualität** nutzen, um zu zeigen, wie man mit Scikit-learn Random Forest Modelle trainiert. Dafür laden wir zunächst das Paket `sklearn.datasets` und daraus die `load_wine` Funktion.

In [3]:
from sklearn.datasets import load_wine

Die `load_wine` Funktion gehört zu den "dataset loaders" und wird genutzt, um kleinere Datensätze (wie unseren hier) zu laden. Wenn wir `return_X_y=True` setzen, bekommen wir zwei getrennte Objekte zurück:

1. Nur die Feature
2. Nur Target/Antwortvariable

In [4]:
features, target = load_wine(return_X_y=True)

## Trainings- und Testsets

Als nächstes wollen wir unsere Daten in **Trainings- und Testsets** aufteilen. Dafür gibt es in `sklearn.model_selection` die `train_test_split` Funktion. Die laden wir wieder ein und geben ihr folgende Argumente mit:

- `features`: die Featurewerte
- `target`: die Antwortvariable
- `test_size`: wie viele der Daten in das Testset sollen (hier 30%)
- `random_state`: Seed für die Pseudozufallsgenerierung von Nummern
- `stratify`: optional, hier sollen Trainings- und Testset die gleichen Proportionen der Klassenverteilungen in unserer Antwortvariablen haben

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    target,
                                                    test_size = 0.30,
                                                    random_state = 42,
                                                    stratify = target)

Der Output von `train_test_split` sind vier Objekte:

- `X_train`: Featurewerte für alle Trainingsinstanzen
- `X_test`: Featurewerte für alle Testinstanzen
- `y_train`: Label/Klassen für alle Trainingsinstanzen
- `y_test`: Label/Klassen für alle Testinstanzen

## Gradient Boosting mit Entscheidungsbäumen

In der vorhergehenden Lektion ging es bereits um Random Forests, eine Methode, die viele Ähnlichkeiten zu Gradient Boosting hat. Denn auch Gradient Boosting wird für überwachtes Machine Learning verwendet, und ist also - wie Random Forests, neuronale Netze und andere Algorithmen - für **Klassifikations- & Regressionsaufgaben** geeignet.

Genau wie Random Forests sind Gradient Boosting Modelle **Ensemble Lerner**, d.h. sie setzen sich aus vielen einzelnen Modellen zusammen - und in der Regel sind diese einzelnen Modelle ebenfalls **Entscheidungsbäume**.

Das Grundprinzip ist also das Gleiche wie bei Random Forests: während einzelne Entscheidungsbäume keine besonders gute Vorhersagekraft haben und schnell zum Overfitting neigen, ist ein Ensemble aus mehreren Entscheidungsbäumen deutlich besser!

Eine detaillierte Erklärung darüber, was Entscheidungsbäume sind und wie sie funktionieren findet ihr in dem oben erwähnten [Kapitel zu Random Forests](https://www.youtube.com/embed/W0ZcjQFTF_g). Darum hier nur kurz die wichtigsten Punkte: Entscheidungsbäume bestehen aus **Knoten oder Ästen**, an denen die **Feature** aufgeteilt werden und die Instanzen entsprechend dieser Aufteilung den Baum **hinunter wandern**. 

Wir haben also eine Kette von **wenn... dann...** Entscheidungen, die unsere Daten immer weiter differenzieren. Diese Differenzierung passiert so lange, bis wir an einem Endpunkt oder **Blatt** angekommen sind. Diese Endpunkte stellen das **Ergebnis**, also bei einer Klassifikation die vorhergesagte Klasse, dar.

Solche Entscheidungsbäume dienen für das Gradient Boosting nun als **Basis-Modell**.

## Boosting

Der Name "Gradient Boosting" beschreibt schon die wesentlichen Funktionen, mit denen diese Methode lernt. Gucken wir uns zunächst das **Boosting** an, mit dem wir einzelne Entscheidungsbäume kombinieren können. 

Im Gegensatz zum Bagging, das bei Random Forests verwendet wird, werden die einzelnen Entscheidungsbäume beim Boosting nicht auf zufälligen Teilmengen der Daten trainiert, sondern auf **gewichteten Teilmengen**. Nach jeder Trainingsrunde, bei der ein Entscheidungsbaum trainiert wurde, werden die Instanzen, deren Vorhersage falsch war, entsprechend des Vorhersagefehlers stärker gewichtet als richtig vorhergesagte Instanzen. Die Daten mit höheren Gewichten, werden entsprechend bevorzugt für die nächsten Teilmengen gesampelt. Die Idee ist, dass **schwierige Fälle besonders genau gelernt werden müssen** und deshalb häufiger gesampelt werden sollen als einfache Fälle.

Das Trainieren auf Teilmengen bezeichnen wir auch als **stochastisches Gradient Boosting**, und hilft dabei, Overfitting zu vermeiden und eine bessere Generalisierbarkeit unseres Modells zu erreichen.

Boosting funktioniert also wie folgt:

1. Wir haben ein **Basis-Modell**, in der Regel Entscheidungsbäume.
2. Mit diesem Basis-Modell machen wir **Vorhersagen** auf unseren Trainingsdaten.
3. Die **Gewichtung** der Instanzen erfolgt nun basierend auf den Vorhersagen. Ist die Vorhersage richtig, bekommt die Instanz ein niedrigeres Gewicht. Ist die Vorhersage falsch, bekommt die Instanz ein höheres Gewicht.
4. Mit diesen Gewichten wird nun eine neue Teilmenge der Daten gesampelt und ein **neuer Entscheidungsbaum wird trainiert**.

## Gradientenoptimierung

Der Gradient kommt nun für die Optimierung ins Spiel. Lernen durch Optimierung mit Gradienten ist uns bereits in der Lektion zu [Neuronalen Netzen und Deep Learning begegnet](https://www.youtube.com/embed/Z42fE0MGoDQ).

Die wichtigsten Aspekte daraus waren, dass wir mit einem neuronalen Netz ein Ergebnis oder eine Vorhersage berechnet haben, die wir mit dem tatsächlichen Ergebnis abgeglichen haben. Aus der **Differenz zwischen Vorhersage und Wirklichkeit** ergab sich ein Fehler, den wir mit einer **Loss-Funktion** dargestellt haben. Diese Loss-Funktion sollte während des Trainingsprozesses **minimiert** werden. Gradienten kamen dabei während dieser Minimierung ins Spiel. Die Methode, die wir uns dabei genauer angesehen haben, hieß **Gradientenabstieg** oder Gradient Descent. Der Gradient ist die **partielle Ableitung** unserer Loss-Funktion und beschreibt also die Fehlerlandschaft. Dort, wo der Gradient am steilsten ist, können wir in der nächsten Trainingsrunde die größte Fehlerminimierung erreichen, das Neuronale Netz lernt also, indem es den Gradienten "hinab steigt".

Mehr dazu in unserem [Video zu Neuronalen Netzen auf YouTube](https://www.youtube.com/embed/Z42fE0MGoDQ).

Gradient Boosting nutzt Gradienten in einer ähnlichen Art wie neuronale Netze, um ebenfalls eine Loss-Funktion zu minimieren. Der große Unterschied ist, dass neuronale Netze den Fehler in einem Modell minimieren, während Gradient Boosting den Fehlern von einem **Ensemble aus Basis-Modellen** minmieren will.
Auch hier nutzen wir unser Basis-Modell zunächst für Vorhersagen auf dem Trainingsset und gleichen diese mit der Wirklichkeit ab, woraus sich der Fehler und der Loss ergibt.
Der Gradient dieser Loss-Funktion wird nun allerdings **zu dem bestehenden Trainingsprozess hinzugefügt**, so dass der nächste Entscheidungsbaum zusätzlich auf diese Gradienten gefittet wird.

## Scikit-learn

Wie trainieren wir nun Gradient Boosting Modelle in Python?

Dafür gibt es wieder mehrere Möglichkeiten und Pakete. Analog zu Random Forests, stelle ich in den Jupyter-Notebooks mit Beispielen, Aufgaben und Lösungen zum einen Scikit-learn vor.

[Scikit-learn](http://scikit-learn.org/stable/) ist eine Machine Learning Bibliothek für Python, die das Trainieren von vielen verschiedenen Algorithmen für Klassifikation, Regression, Clustering, und mehr sehr einfach macht. In Scikit-learn können wir den [Gradient Boosting Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) oder [Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) verwenden.

### Hyperparameter

Da wir mit Entscheidungsbaum-Ensembles und mit Gradienten arbeiten, finden wir in den Hyperparametern, die unseren Lernprozess beschreiben, viele alte Bekannte aus Random Forests und Neuronalen Netzen wieder. 

- Das ist zum einen die Lernrate, also die Schrittgröße für den Gradientenabstieg, und die Schrumpfung oder Shrinkage, mit der wir die Lernrate definieren können.
- Außerdem können wir natürlich die Loss-Funktion wählen,
- die Anzahl an Bäumen für unser Ensemble,
- sowie die Anzahl an Instanzen, die in jedem Blatt vorhanden sein müssen
- und die Baumtiefe und -komplexität.
- Für das Boosting definieren wir den Anteil der gesampelten Instanzen
- und den Anteil der verwendeten Feature.

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(
    learning_rate=0.1,
    loss = "deviance",
    n_estimators = 200,
    min_samples_split = 3,
    min_samples_leaf = 3,
    max_depth = 3,
    random_state = 42
)
gbm.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=3, min_samples_split=3,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              n_iter_no_change=None, presort='auto', random_state=42,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

Wie zuvor können wir die `predict` Funktion verwenden, um unsere Testdaten klassifizieren zu lassen.

In [7]:
y_pred = gbm.predict(X_test)

Evaluieren können wir diese Vorhersagen zum Beispiel mit einer Kreuzmatrix:

In [8]:
from sklearn.metrics import confusion_matrix

unique_label = np.unique(y_test)
pd.DataFrame(confusion_matrix(y_test, y_pred, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pred:{:}'.format(x) for x in unique_label])

,pred:0,pred:1,pred:2
true:0,18,0,0
true:1,1,20,0
true:2,0,0,15


Alternativ zu vorhergesagten Klassen können wir uns mit der `predict_proba` Funktion Vorhersagewahrscheinlichkeiten ausgeben lassen.

In [9]:
gbm.predict_proba(X_test)[0:10]

array([[9.99868601e-01, 5.74801819e-05, 7.39191195e-05],
       [4.34978317e-05, 9.99878647e-01, 7.78552701e-05],
       [9.99787169e-01, 9.72797236e-05, 1.15551236e-04],
       [9.99870065e-01, 5.68394980e-05, 7.30952045e-05],
       [9.97155955e-01, 2.76411783e-03, 7.99275452e-05],
       [9.99819851e-01, 1.00206341e-04, 7.99421802e-05],
       [1.35093150e-04, 6.52984404e-05, 9.99799608e-01],
       [4.64357093e-05, 9.99908393e-01, 4.51716904e-05],
       [5.47128857e-01, 4.21140331e-01, 3.17308125e-02],
       [9.24970432e-05, 3.18824100e-03, 9.96719262e-01]])

## XGBoost

Eine spezielle Implementierung der Gradient Boosting Methode möchte ich hier noch genauer vorstellen: das sogenannte **Extreme Gradient Boosting oder XGBoost**. XGBoost ist besonders beliebt und bekannt worden, weil dieser Algorithmus so gut ist, dass er in vielen Kaggle Competitions zum Sieg geführt hat. XGBoost ist deshalb so erfolgreich, weil er ein paar geniale Tricks verwendet, die das Gradienten Boosting noch effektiver machen.

Einer dieser Tricks ist, dass statt der ersten partiellen Ableitung, die **Gradienten zweiter Ordnung** der Loss-Funktion verwendet werden. Dadurch erhält unser Modell bessere Informationen über die Richtung und die Größe, mit der es den Gradienten hinabsteigen muss, um das Minimum der Loss-Funktion zu finden und die besten Entscheidungsbäume zu approximieren.

XGBoost verwendet außerdem **L1 und L2 Regularisierung**, wodurch es besser in der Lage ist zu generalisieren und Overfitting zu vermeiden.

Außerdem ist XGBoost extrem schnell und kann zusätzlich parallelisiert werden und für verteiltes Trainieren, z.B. auf Spark Clustern verwendet werden.

XGBoost können wir in Python mit dem `xgboost` Paket verwenden. Und zwar entweder als Scikit-learn Implementierung oder nativ.

### Scikit-learn-Implementierung

Für Scikit-learn sieht der Workflow ähnlich aus wie oben:

In [10]:
import xgboost as xgb

In [11]:
xgboost = xgb.XGBClassifier()

In [12]:
xgboost.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [13]:
y_pred = xgboost.predict(X_test)
unique_label = np.unique(y_test)
pd.DataFrame(confusion_matrix(y_test, y_pred, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pred:{:}'.format(x) for x in unique_label])

,pred:0,pred:1,pred:2
true:0,18,0,0
true:1,0,21,0
true:2,0,0,15


### XGBoost-nativ

Das [XGBoost Python-Paket](https://xgboost.readthedocs.io/en/latest/python/index.html) arbeitet mit einer speziellen Datenstruktur, der **DMatrix**. Deshalb wandeln wir unsere Trainings- und Testdaten zunächst in dieses Format um:

In [14]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
test_dmatrix = xgb.DMatrix(data=X_test,label=y_test)

Die einfachste Art mit `xgboost` zu arbeiten, ist die `train` Funktion. Wir definieren [Hyperparameter](https://xgboost.readthedocs.io/en/latest/parameter.html#general-parameters) und füttern sie in die, zusammen mit der DMatrix und der Anzahl an Boosting-Runden in die `train` Funktion:

In [15]:
params = {'eta':0.1, 
         'max_depth':5,
         'silent':1}

bst = xgb.train(dtrain=data_dmatrix, 
                params=params, 
                num_boost_round=50)

Wir können uns während des Trainings die Fehlerwerte ausgeben lassen (hier für Trainings- und Validierungsdaten), indem wir eine **Watchlist** definieren:

In [16]:
watchlist = [(data_dmatrix, 'train'), (test_dmatrix, 'test')]
bst = xgb.train(dtrain=data_dmatrix, 
                params=params, 
                num_boost_round=50,
                evals=watchlist)

[0]	train-rmse:0.79895	test-rmse:0.814
[1]	train-rmse:0.72173	test-rmse:0.739222
[2]	train-rmse:0.651991	test-rmse:0.672404
[3]	train-rmse:0.589008	test-rmse:0.612833
[4]	train-rmse:0.532125	test-rmse:0.559867
[5]	train-rmse:0.480752	test-rmse:0.512924
[6]	train-rmse:0.434354	test-rmse:0.471473
[7]	train-rmse:0.392449	test-rmse:0.435031
[8]	train-rmse:0.354602	test-rmse:0.403149
[9]	train-rmse:0.320419	test-rmse:0.37541
[10]	train-rmse:0.289545	test-rmse:0.351425
[11]	train-rmse:0.261648	test-rmse:0.331023
[12]	train-rmse:0.236439	test-rmse:0.313425
[13]	train-rmse:0.213722	test-rmse:0.297446
[14]	train-rmse:0.193186	test-rmse:0.283696
[15]	train-rmse:0.174585	test-rmse:0.273047
[16]	train-rmse:0.157831	test-rmse:0.263371
[17]	train-rmse:0.142691	test-rmse:0.255207
[18]	train-rmse:0.129019	test-rmse:0.248442
[19]	train-rmse:0.116665	test-rmse:0.242927
[20]	train-rmse:0.105506	test-rmse:0.238074
[21]	train-rmse:0.095428	test-rmse:0.234123
[22]	train-rmse:0.08632	test-rmse:0.230853
[23]	

Alternativ können wir die `cv` Funktion verwenden, um Kreuzvalidierung zu nutzen, hier 3-Fold:

In [17]:
bst_cv = xgb.cv(dtrain=data_dmatrix, 
                params=params, 
                nfold=3,
                num_boost_round=50,
                early_stopping_rounds=10,
                metrics="rmse", 
                as_pandas=True, 
                seed=42)

Hier im Vergleich Trainings- und Validierungs-RMSE (Root Mean Squared Error):

In [18]:
bst_cv.tail()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
45,0.010474,0.001088,0.341533,0.119269
46,0.009609,0.001079,0.341523,0.119392
47,0.008820,0.001065,0.341497,0.119514
48,0.008101,0.001049,0.341515,0.119564
49,0.007447,0.001032,0.341505,0.119599


Evaluieren können wir unser Modell auf den Testdaten:

In [19]:
bst.eval(test_dmatrix)

'[0]\teval-rmse:0.212827'

Oder wir nutzen es für Vorhersagen:

In [20]:
preds_xgb = bst.predict(test_dmatrix)
preds_xgb

array([2.6844833e-03, 9.9383903e-01, 2.6844833e-03, 2.6844833e-03,
       2.4542863e-02, 2.6844833e-03, 1.9906461e+00, 9.9727887e-01,
       9.6621418e-01, 1.9906461e+00, 9.9603283e-01, 9.9666268e-01,
       1.9906461e+00, 9.9865365e-01, 5.5614077e-03, 1.9906461e+00,
       8.5669547e-01, 9.6929204e-01, 1.9906461e+00, 1.9906461e+00,
       9.9789667e-01, 1.9906461e+00, 1.9906461e+00, 1.9906461e+00,
       9.9319249e-01, 1.9906461e+00, 2.6844833e-03, 2.5594944e-01,
       2.6844833e-03, 9.5309651e-01, 9.5150578e-01, 8.5669547e-01,
       1.9906461e+00, 1.0012242e+00, 9.7202969e-01, 1.9906461e+00,
       9.9319249e-01, 9.9772471e-01, 1.0010464e+00, 2.6844833e-03,
       1.9906461e+00, 2.6844833e-03, 2.6844833e-03, 2.6844833e-03,
       2.6844833e-03, 9.9543166e-01, 9.9707168e-01, 2.6844833e-03,
       1.9906461e+00, 1.3731916e-03, 9.9782139e-01, 9.9860030e-01,
       1.9906461e+00, 2.6844833e-03], dtype=float32)

In [21]:
labels = test_dmatrix.get_label()
labels

array([0., 1., 0., 0., 0., 0., 2., 1., 1., 2., 1., 1., 2., 1., 0., 2., 1.,
       0., 2., 2., 1., 2., 2., 2., 1., 2., 0., 1., 0., 1., 0., 1., 2., 1.,
       1., 2., 1., 1., 1., 0., 2., 0., 0., 0., 0., 1., 1., 0., 2., 0., 1.,
       1., 2., 0.], dtype=float32)

## Gradient Boosting mit H2O

Auch in H2O finden wir eine Implementierung für Gradient Boosting. [H2O](https://www.h2o.ai/) ist eine beliebte Open-Source Machine Learning Plattform, mit der man ebenfalls sehr einfach und schnell Random Forests trainieren und tunen kann.

Im Unterschied zu Scikit-learn läuft H2O auf Clustern mit einem Java Backend. Wir können unser Cluster lokal starten oder z.B. mit Spark, Hadoop oder anderen Umgebungen. Je nachdem, wie rechenintensiv unsere Modelle sind, können wir durch das Nutzen von mehreren Clustern und Knoten eine deutlich schnellere Trainingszeit erreichen als mit Scikit-learn.

Zunächst importieren wir die `h2o` Bibliothek und starten das Cluster. Mit `nthreads = -1` werden alle Kerne einer Machine genutzt.

In [22]:
# Load the H2O library and start up the H2O cluster locally on your machine
import h2o

# Number of threads, nthreads = -1, means use all cores on your machine
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_181"; OpenJDK Runtime Environment (build 1.8.0_181-8u181-b13-2~deb9u1-b13); OpenJDK 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /usr/local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpya5c3rm9
  JVM stdout: /tmp/tmpya5c3rm9/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpya5c3rm9/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,7 days and 3 hours
H2O cluster name:,H2O_from_python_unknownUser_3k9t60
H2O cluster total nodes:,1
H2O cluster free memory:,444.5 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [23]:
data = load_wine()
df = pd.DataFrame(data.data, columns=data.feature_names)
target = pd.DataFrame({'class':data.target})

df_c = pd.concat([target, df], axis=1)
df_c.head()

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,0,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,0,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,0,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,0,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


Bevor wir mit `h2o` arbeiten können, müssen wir unsere Daten in ein H2O Frame konvertieren. Dafür gibt es die `h2o.H2OFrame` Funktion. Unsere Antwortvariable ist im Moment noch numerisch, h2o würde also eine Regression durchführen. Wir wollen aber eine Klassifikation trainieren, darum konvertiere ich die Variable mit `asfactor`.

In [24]:
hf = h2o.H2OFrame(df_c)
hf[0] = hf[0].asfactor()  
hf[0].isfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


[True]

Auch in `h2o` können wir unsere Daten sehr einfach in Trainings- und Testsets einteilen. Zu Demonstrationszwecken teile ich die Daten hier in drei Sets ein: Training, Validierung und Test. Bei der geringen Anzahl an Instanzen ist das aber eigentlich nicht sinnvoll - wir würden stattdessen Kreuzvalidierung verwenden.

Die `split_frame` Funktion bekommt einen Vektor von Werten zwischen 0 und 1, die in Summe weniger als 1 sein müssen. Diese Werte stellen den Anteil der Daten für das Trainingsset (hier 70%), das Validierungsset (hier 15%) und das Testset (die verbleibenden 15%) dar. Die drei Datensets bekommen wir als separate Objekte zurück.

In [25]:
train, valid, test = hf.split_frame([0.7, 0.15], seed=42)

Zur Vorbereitung definieren wir nun auch noch einen Vektor mit allen Featurenamen (Spaltennamen), die wir zum Trainieren verwenden wollen:

In [26]:
hf_X = hf.col_names[1:len(hf.col_names)]
hf_X

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

Außerdem generieren wir noch einen mit der Antwortvariablen:

In [27]:
hf_y = hf.col_names[0]
hf_y

'class'

Die entsprechende Funktion hier heißt [H2O Gradient Boosting Estimator](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/gbm.html).

In [28]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [29]:
gbm_h2o = H2OGradientBoostingEstimator(
    max_depth = 3,
    min_rows = 3,
    seed = 42
)

Mit `train` erfolgt dann das eigentliche Traineren des Modells. Wenn wir mit einem Validierungsset arbeiten, wie hier, geben wir dieses neben dem Trainingsset an. Falls wir kein Validierungsset hätten und Kreuzvalidierung durchführen wollten, würden wir statt `validation_frame` die Argumente `nfolds` (Anzahl Folds) im H2ORandomForestEstimator setzen. Die Namen der Spalten für Feature und Antwortvariablen geben wir mit `x` und `y`.

In [30]:
gbm_h2o.train(x = hf_X, 
         y = hf_y, 
         training_frame = train, 
         validation_frame = valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


Eine Übersicht über die Performance unseres Modells auf den Testdaten erhalten wir mit der `model_performance` Funktion:

In [31]:
performance = gbm_h2o.model_performance(test_data=test)
print(performance)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.024284013580955083
RMSE: 0.1558332877820239
LogLoss: 0.0688345169342984
Mean Per-Class Error: 0.06666666666666667
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,Error,Rate
9.0,0.0,0.0,0.0,0 / 9
0.0,10.0,0.0,0.0,0 / 10
0.0,1.0,4.0,0.2,1 / 5
9.0,11.0,4.0,0.0416667,1 / 24


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9583333
2,1.0
3,1.0


## XGBoost mit H2O

Auch in H2O finden wir eine Implementierung von XGBoost.

Hier heißt die Funktion entsprechend [H2O XGBoost Estimator](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/xgboost.html).

In [32]:
from h2o.estimators import H2OXGBoostEstimator

In [33]:
xgb_h2o = H2OXGBoostEstimator(
    max_depth = 3,
    min_rows = 3,
    seed = 42
)

In [34]:
xgb_h2o.train(x = hf_X, 
         y = hf_y, 
         training_frame = train, 
         validation_frame = valid)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [35]:
performance_xgb = xgb_h2o.model_performance(test_data=test)
print(performance_xgb)


ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.030792554899798635
RMSE: 0.17547807526810474
LogLoss: 0.11250859163847905
Mean Per-Class Error: 0.06666666666666667
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



0,1,2,Error,Rate
9.0,0.0,0.0,0.0,0 / 9
0.0,10.0,0.0,0.0,0 / 10
0.0,1.0,4.0,0.2,1 / 5
9.0,11.0,4.0,0.0416667,1 / 24


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9583333
2,1.0
3,1.0


Gradient Boosting ist außerdem in [H2O's AutoML Funktion](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html) enthalten. AutoML steht für automatisches Machine Learning und vergleicht mit einer Funktion mehrere Algorithmen, wie Gradient Boosting, Random Forests und Neuronale Netze, sowie verschiedene Hyperparameter.